# このNotebookについて
- 参考: [Link](https://ryono-blog.com/%E3%80%90python%E3%80%91%E9%81%BA%E4%BC%9D%E7%9A%84%E3%82%A2%E3%83%AB%E3%82%B4%E3%83%AA%E3%82%BA%E3%83%A0%EF%BC%88ga%EF%BC%89%E3%81%A7%E3%82%B9%E3%82%B1%E3%82%B8%E3%83%A5%E3%83%BC%E3%83%AA%E3%83%B3)

# 問題定義
- 1週間のシフトを5人で回す。
- 出勤が1で休暇は0。

| 曜日   | 月 | 火 | 水 | 木 | 金 | 土 | 日 |
|------|---|---|---|---|---|---|---|
| 必要人数 | 2 | 2 | 2 | 1 | 2 | 3 | 3 |
| 従業員１  | 1 | 0 | 0 | 0 | 0 | 1 | 1 |
| 従業員2  | 0 | 1 | 0 | 0 | 1 | 0 | 1 |
| 従業員3  | 0 | 1 | 1 | 0 | 0 | 1 | 0 |
| 従業員4  | 1 | 0 | 0 | 1 | 0 | 0 | 1 |
| 従業員5  | 0 | 0 | 1 | 0 | 1 | 1 | 0 |

- 出力は二次元配列となる。  
`[[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0], [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0], [0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0], [0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0]]`

- 制約条件
  - 各従業員は出勤が3日まで。
  - 必要人数と実際に割り振る人数が一致する。

# ライブラリインポート

In [57]:
from datetime import datetime
import random
from decimal import Decimal
import numpy as np
from itertools import zip_longest

# 定義

## パラメータ

In [58]:
# シフトの定義（シフト, 必要人数）
SHIFT_BOX = [('Mon',2), ('Tue',2), ('Wed',2), ('Thu',1), ('Fri',2), ('Sat',3), ('Sun',3)]
# シフト
DAY = len(SHIFT_BOX)
# 従業員数
PEOPLE = 5
# 各従業員の出勤最大日数
MAX_SHIFT = 3
# 遺伝子情報の長さ
GENOM_LENGTH = PEOPLE * DAY
# 遺伝子集団の大きさ
MAX_GENOM_LIST = 300
# 遺伝子選択数
SELECT_GENOM = 40
# 個体突然変異確率
INDIVIDUAL_MUTATION = 0.1
# 遺伝子突然変異確率
GENOM_MUTATION = 0.1
# 繰り返す世代数
MAX_GENERATION = 40
# 繰り返しをやめる評価値の閾値
THRESSHOLD = 0

## 染色体クラス
遺伝子情報や、適応度を格納するクラス

In [59]:
# 染色体の定義
class Chromosome:

    genom = None
    evaluation = None

    def __init__(self, genom, evaluation) -> None:
        self.genom = genom
        self.evaluation = evaluation

    def getGenom(self):
        return self.genom

    def getEvaluation(self):
        return self.evaluation

    def setGenom(self, genom_list) -> None:
        self.genom = genom_list

    def setEvaluation(self, evaluation) -> None:
        self.evaluation = evaluation

## 複数個体を生成する関数

In [60]:
def create_Chromosome(length):
    """
    引数で指定された桁のランダムな遺伝子情報を生成、格納したChromosomeClassで返す。
    :param length: 遺伝子情報長
    :return: 生成した個体集団ChromosomeClass
    """
    days_list = []
    genom_list = []
    
    for i in range(int(GENOM_LENGTH/DAY)): # genom長は人の数x日数
        for j in range(DAY):
            days_list.append(float(random.randint(0,1)))
        genom_list.append(days_list)
        days_list = []
    return Chromosome(genom_list, 0)

## 評価関数
どれだけ制約が守られているかを評価する。

In [61]:
def evaluation(Chromosome):
    """評価関数。制約式のペナルティーを適応度とする
    :param Chromosome: 評価を行うChromosomeClass
    :return: 評価処理をしたChromosomeClassを返す
    """
    fitness = constraints(Chromosome)
    
    return fitness

def constraints(Chromosome):
    """制約関数。制約が満たされない場合、ペナルティーを付与する。
    :param Chromosome: 評価を行うChromosomeClass
    :return: penalty 
    """
    global penalty
    penalty = 0.0
    # 多次元配列から行列に変換
    genom_arr = np.array(Chromosome.getGenom())
    # 各従業員の出勤をMAX_SHIFT（3日）までに抑える制約
    for i in range(genom_arr.shape[0]):
        employee = genom_arr[i]
        if sum(employee) > MAX_SHIFT:
            penalty += 50.0 * abs(sum(employee) - MAX_SHIFT)
    # 必要人数とアサイン人数が一致するようにする制約
    for i in range(genom_arr.shape[1]):
        if sum([shift[i] for shift in genom_arr]) != SHIFT_BOX[i][1]:
            penalty += 10.0 * abs(sum([shift[i] for shift in genom_arr]) - SHIFT_BOX[i][1])
                                       
    return penalty

## 選択
今回はエリート選択を利用する。

In [62]:
def elite_select(Chromosome, elite_length):
    """選択関数です。エリート選択
    評価が高い順番にソートを行った後、一定以上の染色体を選択
    :param Chromosome: 選択を行うChromosomeClassの配列
    :param elite_length: 選択する染色体数
    :return: 選択処理をした一定のエリート、ChromosomeClassを返す
    """
    # 現行世代個体集団の評価を低い順番にソートする
    sort_result = sorted(Chromosome, reverse=False, key=lambda u: u.evaluation)
    # 一定の上位を抽出する
    result = [sort_result.pop(0) for i in range(elite_length)]
    return result

## 交叉

### 二点交叉

In [63]:
def crossover(Chromosome_one, Chromosome_second):
    """交叉関数。二点交叉
    :param Chromosome: 交叉させるChromosomeClassの配列
    :param Chromosome_one: 一つ目の個体
    :param Chromosome_second: 二つ目の個体
    :return: 二つの子孫ChromosomeClassを格納したリスト返す
    """
    # 子孫を格納するリストを生成
    genom_list = []
    # 入れ替える二点の点を設定
    cross_one = random.randint(0, int(GENOM_LENGTH/DAY))
    cross_second = random.randint(cross_one, int(GENOM_LENGTH/DAY))
    # 遺伝子を取り出し
    one = Chromosome_one.getGenom()
    second = Chromosome_second.getGenom()
    # 交叉
    progeny_one = one[:cross_one] + second[cross_one:cross_second] + one[cross_second:]
    progeny_second = second[:cross_one] + one[cross_one:cross_second] + second[cross_second:]
    # ChromosomeClassインスタンスを生成して子孫をリストに格納
    genom_list.append(Chromosome(progeny_one, 0))
    genom_list.append(Chromosome(progeny_second, 0))
    return genom_list

### 一様交叉

In [64]:
def uniform_crossover(Chromosome_one, Chromosome_second):
    """交叉関数。一様交叉。
    :param Chromosome: 交叉させるChromosomeClassの配列
    :param Chromosome_one: 一つ目の個体
    :param Chromosome_second: 二つ目の個体
    :return: 二つの子孫ChromosomeClassを格納したリスト返す
    """
    # 子孫を格納するリストを生成
    genom_list = []
    # 遺伝子を取り出す
    one = Chromosome_one.getGenom()
    second = Chromosome_second.getGenom()
    # 交叉
    for i in range(len(one)):
        if np.random.rand() < 0.5:
            genom_list.append(one[i])
        else:
            genom_list.append(second[i])
            
    return [Chromosome(genom_list, 0)]

### 突然変異

In [65]:
def mutation(Chromosome, individual_mutation, genom_mutation):
    """突然変異関数。
    :param Chromosome: 突然変異をさせるChromosomeClass
    :param individual_mutation: 固定に対する突然変異確率
    :param Chromosome_mutation: 遺伝子一つ一つに対する突然変異確率
    :return: 突然変異処理をしたgenomClassを返す"""
    Chromosome_list = []
    for genom in Chromosome:
        # 個体に対して一定の確率で突然変異が起きる
        if individual_mutation > (random.randint(0, 100) / Decimal(100)):
            genom_list = []
            for i_ in genom.getGenom():
                ga_list = []
                # 個体の遺伝子情報一つ一つに対して突然変異が起こる
                if genom_mutation > (random.randint(0, 100) / Decimal(100)):
                    for j in range(len(i_)):
                        ga_list.append(float(random.randint(0, 1)))
                    genom_list.append(ga_list)
                else:
                    genom_list.append(i_)
            genom.setGenom(genom_list)
            Chromosome_list.append(genom)
        else:
            Chromosome_list.append(genom)
    return Chromosome_list

## 世代交代

In [66]:
def next_generation_gene_create(Chromosome, Chromosome_elite, Chromosome_progeny):
    """
    世代交代処理
    :param Chromosome: 現行世代個体集団
    :param Chromosome_elite: 現行世代エリート集団
    :param Chromosome_progeny: 現行世代子孫集団
    :return: 次世代個体集団
    """
    # 現行世代個体集団の評価を高い順番にソート
    next_generation_geno = sorted(Chromosome, reverse=True, key=lambda u: u.evaluation)
    # 追加するエリート集団と子孫集団の合計分を取り除く
    for i in range(0, len(Chromosome_elite) + len(Chromosome_progeny)):
        next_generation_geno.pop(0)
    # エリート集団と子孫集団を次世代集団を次世代へ追加
    next_generation_geno.extend(Chromosome_elite)
    next_generation_geno.extend(Chromosome_progeny)
    return next_generation_geno

# メインループ

In [67]:
# 一番最初の現行世代個体集団を生成
current_generation_individual_group = []
for i in range(MAX_GENOM_LIST):
    current_generation_individual_group.append(create_Chromosome(GENOM_LENGTH))

for count_ in range(1, MAX_GENERATION + 1):
    # 現行世代個体集団の遺伝子を評価し、ChromosomeClassに代入
    for i in range(MAX_GENOM_LIST):
        evaluation_result = evaluation(current_generation_individual_group[i])
        current_generation_individual_group[i].setEvaluation(evaluation_result)
    # エリート個体を選択
    elite_genes = elite_select(current_generation_individual_group,SELECT_GENOM)
    # エリート遺伝子を交叉させ、リストに格納
    progeny_gene = []
    for i in range(0, SELECT_GENOM):
        progeny_gene.extend(crossover(elite_genes[i - 1], elite_genes[i]))
    # 次世代個体集団を現行世代、エリート集団、子孫集団から作成
    next_generation_individual_group = next_generation_gene_create(current_generation_individual_group,
                                                                   elite_genes, progeny_gene)
    # 次世代個体集団全ての個体に突然変異を施す
    next_generation_individual_group = mutation(next_generation_individual_group,INDIVIDUAL_MUTATION,GENOM_MUTATION)

    # 1世代の進化的計算終了

    # 各個体適用度を配列化
    fits = [i.getEvaluation() for i in current_generation_individual_group]

    # 進化結果を評価
    min_ = min(fits)
    max_ = max(fits)
    avg_ = Decimal(sum(fits)) / Decimal(len(fits))

    # 現行世代の進化結果を出力します
    print(datetime.now(),
          f'世代数 : {count_}  ',
          f'Min : {min_:.3f} ',
          f'Max : {max_:.3f}  ',
          f'Avg : {avg_:.3f}  '
         )
    # 現行世代と次世代を入れ替える
    current_generation_individual_group = next_generation_individual_group
    # 適応度が閾値に達したら終了
    if THRESSHOLD >= min_:
        print('optimal')
        print(datetime.now(),
          f'世代数 : {count_}  ',
          f'Min : {min_:.3f} ',
          f'Max : {max_:.3f}  ',
          f'Avg : {avg_:.3f}  '
         )
        break
# 最最良個体結果出力
print(f'最良個体情報:{elite_genes[0].getGenom()}')

2024-06-29 14:00:55.134698 世代数 : 1   Min : 40.000  Max : 580.000   Avg : 263.267  
2024-06-29 14:00:55.142318 世代数 : 2   Min : 30.000  Max : 320.000   Avg : 157.300  
2024-06-29 14:00:55.151307 世代数 : 3   Min : 30.000  Max : 200.000   Avg : 97.167  
2024-06-29 14:00:55.156157 世代数 : 4   Min : 30.000  Max : 270.000   Avg : 66.867  
2024-06-29 14:00:55.168050 世代数 : 5   Min : 20.000  Max : 200.000   Avg : 45.867  
2024-06-29 14:00:55.172146 世代数 : 6   Min : 20.000  Max : 200.000   Avg : 37.133  
2024-06-29 14:00:55.172146 世代数 : 7   Min : 20.000  Max : 190.000   Avg : 37.367  
2024-06-29 14:00:55.188388 世代数 : 8   Min : 20.000  Max : 380.000   Avg : 41.367  
2024-06-29 14:00:55.188388 世代数 : 9   Min : 20.000  Max : 270.000   Avg : 36.900  
2024-06-29 14:00:55.201196 世代数 : 10   Min : 0.000  Max : 160.000   Avg : 37.400  
optimal
2024-06-29 14:00:55.201196 世代数 : 10   Min : 0.000  Max : 160.000   Avg : 37.400  
最良個体情報:[[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0], [0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0], [1.0, 